# 1. 모듈 임포트

In [33]:
#기본모듈
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

#전처리 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

#머신러닝 모델 라이브러리
from sklearn.linear_model import LogisticRegression
import joblib
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#평가지표
from sklearn.metrics import accuracy_score, recall_score, precision_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import classification_report

#사용할 영상처리 및 골격인식 라이브러리
import cv2
import mediapipe as mp

### ⇓ 설치 및 사용 모듈 버전 ⇓

python : 3.9.7  
pandas :  1.3.3  
numpy  :  1.19.5  
scikit-learn  :  0.24.2  
joblib        :  1.0.1  
tensorflow    :  2.7.0  
mediapipe     :  0.8.9  
opencv-python :  4.5.3.56  
opencv-contrib-python  :  4.5.4.58  
imbalanced-learn       :  0.8.1  
imblearn               :  0.0  
~~keras  :  2.7.0~~  
~~tensorflow-cpu  :  2.6.0  
tensorflow-estimator  :  2.7.0  
tensorflow-io-gcs-filesystem  :  0.22.0~~

# 2. 함수 정의

#### 2-1. 운동 횟수 카운팅 모델 생성 함수

In [2]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(64, 64, 3))) # 간단한 훈련 과정이기에 입력의 크기를 64x64로 설정하였습니다
    
    # 합성곱 층 Conv2D와 MaxPooling을 반복해서 사용하여 인공신경망의 층을 구성하였습니다.
    model.add(tf.keras.layers.Conv2D(6, (3, 3), padding='same', activation='relu')) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # 과적합을 방지하고자 Dropout 함수를 사용해서 인풋 데이터의 25%의 노드들을 무작위로 0으로 만들었습니다
    model.add(tf.keras.layers.Dropout(0.25))

    # Flatten으로 행렬을 배열로 바꾸었고
    # Dense를 통해 인풋을 넣었을 때, 아웃풋으로 바꾸어 주는 중간다리 역할을 하게끔 하였습니다
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Activation('softmax'))
    # 그리고 마지막 활성화 함수를 softmax로 하여 확률값을 뱉어주게끔 하였습니다
    
    return model

#### 2-2. 각도 계산 함수

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0]) # 해당 값들의 역탄젠트 처리로 라디안 계산
    angle = np.abs(radians * 180.0 / np.pi) # 라디안으로 각도 계산

    if angle > 180.0: # 각도값을 0~180으로 맞춰주는
        angle = 360 - angle

    return angle

#### 2-3. 파일 이미지 전처리 및 각도 및 라벨 데이터 수집 함수

In [4]:
def cal_in_file(df, p, label, excercise):
    path = './data/mp_train/'  # 데이터의 경로 지정
    path= path + p + '/'
    file_list = os.listdir(path)
    file_list_py = [file for file in file_list if file.endswith('.png') or file.endswith('.PNG') or file.endswith('.jpg')] # 해당 경로의 파일들을 목록화
    
    mp_drawing = mp.solutions.drawing_utils  # 미디어 파이프의 API를 사용해서 인체 골격을 인식하고
    mp_pose = mp.solutions.pose              # 그려주는 모듈입니다
        
    for i in file_list_py:
        image = cv2.imread(path+i, cv2.IMREAD_COLOR) # 해당 경로의 이미지 파일을 컬러로 읽어들입니다

        # 미디어 파이프 요소들을 세팅합니다
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 일반적으로 이미지를 imread로 읽으면 BGR로 받아들이기 때문에 
            image.flags.writeable = False                 # mediapipe에서 적용하기 위해 RGB로 바꾸고, 영상에서 다른 작업을 못하도록 설정합니다

            # 포즈를 검출합니다
            results = pose.process(image)

            # 다시 BGR로 바꿔줍니다
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 랜드마크를 검출합니다. 인체골격을 각 지점마다 따올 수가 있습니다.
            try:
                landmarks = results.pose_landmarks.landmark

                # 좌측 지표값
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                # 좌측 각도계산
                left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                left_waist_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # 우측 지표값
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                # 우측 각도계산
                right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                right_waist_angle = calculate_angle(right_shoulder, right_hip, right_knee)
                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)


            except:
                pass


            # 검출한 랜드마크들을 실제 영상에 그려줍니다
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                      )
            # 데이터 프레임으로 구성하기 위해 다음과 같은 작업을 합니다.
            if excercise==1:
                k = pd.DataFrame(data=[[left_elbow_angle, left_shoulder_angle, right_elbow_angle, right_shoulder_angle, label]], 
                             columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨', '점수'])
            else:
                k = pd.DataFrame(data=[[left_waist_angle, left_knee_angle, right_waist_angle, right_knee_angle, label]], 
                         columns=['왼쪽 허리', '왼쪽 무릎', '오른쪽 허리', '오른쪽 무릎', '점수'])
                
            df=df.append(k)
            df = df.reset_index(drop=True)
    return df

# 3. 운동 횟수 카운팅 모델 학습

#### -> 모델 생성 및 저장

In [5]:
# Data Augmentation을 위한 데이터 생성(generator)하는 방법으로 keras의 ImageDataGenerator 사용
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# 디렉토리 path를 설정하고 증강 데이터 배치를 생성하는 flow_from_directory 함수를 사용
train_generator = train_datagen.flow_from_directory(
    './data/train',
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
    seed=42)
validation_generator = test_datagen.flow_from_directory(
    './data/validation',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical')

# 모델 생성 함수 호출하여 모델 생성
model = create_model()

# 모델의 optimizer와 loss, metrics 선택
# 3개 이상의 클래스를 분류하는 multiclassification이므로 CategoricalCrossentropy 적용
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy'])

# 모델 학습
# 학습 데이터 반복 횟수를 10으로 설정
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    validation_freq=1
)

model.save('model.h5', overwrite=True)

Found 376 images belonging to 3 classes.
Found 84 images belonging to 3 classes.
Epoch 1/10
24/24 [==============================] - 8s 306ms/step - loss: 1.0080 - categorical_accuracy: 0.4787 - val_loss: 0.6701 - val_categorical_accuracy: 0.8452
Epoch 2/10
24/24 [==============================] - 5s 188ms/step - loss: 0.5551 - categorical_accuracy: 0.7766 - val_loss: 0.3024 - val_categorical_accuracy: 0.8452
Epoch 3/10
24/24 [==============================] - 4s 183ms/step - loss: 0.4484 - categorical_accuracy: 0.8351 - val_loss: 0.3457 - val_categorical_accuracy: 0.8690
Epoch 4/10
24/24 [==============================] - 4s 184ms/step - loss: 0.4997 - categorical_accuracy: 0.8351 - val_loss: 0.2721 - val_categorical_accuracy: 0.9167
Epoch 5/10
24/24 [==============================] - 4s 185ms/step - loss: 0.3613 - categorical_accuracy: 0.8830 - val_loss: 0.2403 - val_categorical_accuracy: 0.9167
Epoch 6/10
24/24 [==============================] - 4s 185ms/step - loss: 0.3807 - catego

# 4. 파일 접근 및 데이터 프레임 구성 (운동 각도)

### 4-1. mp_train/push_up/ 의 파일들 각도 계산

##### 4-1-1. down_correct

In [5]:
df = pd.DataFrame(columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨'])
df = cal_in_file(df, 'push_up/down_correct', 1, 1)
df

,왼쪽 팔꿈치,왼쪽 어깨,오른쪽 팔꿈치,오른쪽 어깨,점수
0,36.094779,5.313295,20.662132,66.933597,1.0
1,41.106454,3.425422,15.317009,75.770951,1.0
2,42.467221,0.600137,11.236330,73.744296,1.0
3,55.117431,16.374145,9.676304,65.725450,1.0
4,39.050797,5.651023,11.003275,69.046638,1.0
5,41.166219,1.474979,13.780750,73.694411,1.0
6,34.780610,12.848503,7.391771,63.785820,1.0
7,99.068245,32.589993,137.016088,7.617493,1.0
8,103.837773,29.294328,135.300784,5.301512,1.0
9,103.821500,24.160785,134.043963,4.984872,1.0


##### 4-1-2. down_wrong

In [6]:
df = cal_in_file(df, 'push_up/down_wrong', 0, 1)
df

,왼쪽 팔꿈치,왼쪽 어깨,오른쪽 팔꿈치,오른쪽 어깨,점수
0,36.094779,5.313295,20.662132,66.933597,1.0
1,41.106454,3.425422,15.317009,75.770951,1.0
2,42.467221,0.600137,11.236330,73.744296,1.0
3,55.117431,16.374145,9.676304,65.725450,1.0
4,39.050797,5.651023,11.003275,69.046638,1.0
...,...,...,...,...,...
216,95.902320,60.201069,120.024581,147.223990,0.0
217,136.662627,93.407173,123.641020,169.443325,0.0
218,151.477693,83.322559,159.778397,137.286583,0.0
219,115.726728,103.589602,81.407360,162.917310,0.0


### 4-2. mp_train/squat/ 의 파일들 각도 계산

##### 4-2-1. down_correct

In [7]:
df2=pd.DataFrame(columns=['왼쪽 허리', '왼쪽 무릎', '오른쪽 허리', '오른쪽 무릎'])
df2 = cal_in_file(df2, 'squat/down_correct', 1, 2)
df2

,왼쪽 허리,왼쪽 무릎,오른쪽 허리,오른쪽 무릎,점수
0,54.217243,57.855414,51.943992,56.586952,1.0
1,54.238452,58.994730,52.718187,58.101249,1.0
2,50.574233,54.397550,49.018532,55.227366,1.0
3,49.578736,56.580298,47.866487,56.333156,1.0
4,49.068880,53.955939,47.796407,55.105234,1.0
...,...,...,...,...,...
158,106.178215,103.194227,112.116346,95.109191,1.0
159,155.234128,160.016123,147.498188,154.530780,1.0
160,178.849053,177.557547,123.499556,109.838759,1.0
161,73.683770,73.892401,76.962375,62.166687,1.0


##### 4-2-2. down_wrong

In [8]:
df2 = cal_in_file(df2, 'squat/down_wrong', 0, 2)
df2

,왼쪽 허리,왼쪽 무릎,오른쪽 허리,오른쪽 무릎,점수
0,54.217243,57.855414,51.943992,56.586952,1.0
1,54.238452,58.994730,52.718187,58.101249,1.0
2,50.574233,54.397550,49.018532,55.227366,1.0
3,49.578736,56.580298,47.866487,56.333156,1.0
4,49.068880,53.955939,47.796407,55.105234,1.0
...,...,...,...,...,...
193,179.774603,172.364966,128.729696,159.579861,0.0
194,173.175963,178.995848,137.070453,162.244845,0.0
195,173.493769,176.839678,133.142918,157.468925,0.0
196,175.190126,178.373769,122.858585,158.198323,0.0


# 5. 운동 수행 가이드 모델 학습 (Logistic Regression)

### -> 팔굽혀펴기(df)  &  스쿼트(df2)

In [39]:
X=df.drop(['점수'], axis=1)
y=df['점수']   # 팔굽

X2=df2.drop(['점수'], axis=1)
y2=df2['점수'] # 스쿼트

In [41]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=3/10, stratify=y, random_state=12124343) # 팔굽
X_train2, X_test2, y_train2, y_test2=train_test_split(X2,y2, test_size=3/10, stratify=y2, random_state=12124344) # 스쿼트

In [23]:
LR=LogisticRegression(penalty='none', solver='sag', max_iter=5000, tol=1e-4)  # 팔굽
LR2=LogisticRegression(penalty='none', solver='sag', max_iter=5000, tol=1e-4) # 스쿼트
# solver에 gradient descent 같은 알고리즘을 넣는 것
# LR=LogisticRegression(penalty='l2', solver='newtone-cg', max_iter=3000, tol=1e-4, C=1.5)
# Newton-CG methods use a Conjugate Gradient

In [24]:
LR.fit(X_train, y_train)    # 팔굽
LR2.fit(X_train2, y_train2) # 스쿼트

LogisticRegression(max_iter=5000, penalty='none', solver='sag')

# 6. 성능 지표

In [25]:
y_test_pred=LR.predict(X_test)    # 팔굽
y_test_pred2=LR2.predict(X_test2) # 스쿼트

In [26]:
print('팔굽혀펴기')
print('- Accuracy(Test) : {:.4}'.format(accuracy_score(y_test, y_test_pred)))
print('- precision score(Test) : {:.4}'.format(precision_score(y_test, y_test_pred, pos_label=0))) # positive 라벨이 무엇인지 지정해준 것
print('- recall score(Test) : {:.4}'.format(recall_score(y_test, y_test_pred, pos_label=0))) # 지정하지 않으면 1로 지정되어서
print('- F1 score(Test) : {:.4}'.format(f1_score(y_test, y_test_pred, pos_label=0))) # 에러가 나올 걸
print('- balanced score(Test) : {:.4}'.format(balanced_accuracy_score(y_test, y_test_pred)))

print()
print()
print()

print('스쿼트')
print('- Accuracy(Test) : {:.4}'.format(accuracy_score(y_test2, y_test_pred2)))
print('- precision score(Test) : {:.4}'.format(precision_score(y_test2, y_test_pred2, pos_label=0))) # positive 라벨이 무엇인지 지정해준 것
print('- recall score(Test) : {:.4}'.format(recall_score(y_test2, y_test_pred2, pos_label=0))) # 지정하지 않으면 1로 지정되어서
print('- F1 score(Test) : {:.4}'.format(f1_score(y_test2, y_test_pred2, pos_label=0))) # 에러가 나올 걸
print('- balanced score(Test) : {:.4}'.format(balanced_accuracy_score(y_test2, y_test_pred2)))

팔굽혀펴기
- Accuracy(Test) : 0.7778
- precision score(Test) : 0.7778
- recall score(Test) : 1.0
- F1 score(Test) : 0.875
- balanced score(Test) : 0.5



스쿼트
- Accuracy(Test) : 0.9167
- precision score(Test) : 0.875
- recall score(Test) : 0.6364
- F1 score(Test) : 0.7368
- balanced score(Test) : 0.808


# 7. 모델 저장

In [70]:
joblib.dump(LR, 'pushup.pkl')
joblib.dump(LR2, 'squat.pkl')

['squat.pkl']

# 8. 모델 불러오기

In [71]:
LR = joblib.load('pushup.pkl') 
LR2 = joblib.load('squat.pkl') 

In [77]:
print(LR.predict([[23, 124, 412, 1], [21, 123, 4, 12]]))
print(LR2.predict([[23, 124, 412, 1], [21, 123, 4, 12]]))

[1. 1.]
[1. 1.]


In [76]:
if LR.predict([[23, 124, 412, 1]])==0:
    print('팔굽 더 숙여야')
elif LR.predict([[23, 124, 412, 1]])==1:
    print('팔굽 잘한')
    
if LR2.predict([[23, 124, 412, 1]])==0:
    print('스쿼트 무릎을 더 구부리거나 허리를 좀 펴야')
elif LR2.predict([[23, 124, 412, 1]])==1:
    print('스쿼트 잘한')

팔굽 잘한
스쿼트 잘한


# <메인 시뮬레이션 화면>

In [1]:
mp_drawing = mp.solutions.drawing_utils # 미디어 파이프의 API를 사용해서 인체 골격을 인식하고
mp_pose = mp.solutions.pose             # 그려주는 모듈입니다

LR = joblib.load('pushup.pkl') # 모델 불러오기
LR2 = joblib.load('squat.pkl') 

model = tf.keras.models.load_model('model.h5') # 모델 불러오기
cap = cv2.VideoCapture(0) # OpenCV의 웹캠을 불러들이는 모듈

# '팔굽혀펴기' 혹은 '스쿼트' 구분을 위한 변수들
ex = 1
flag = False
flag2 = False

# 미디어파이프 요소들을 설정
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    a = []
    b = []
    c = []
    d = []

    ret, frame1 = cap.read() # 웹캠에서 영상 획득
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY) # 프레임을 명암도 영상으로 변환
    hsv = np.zeros_like(frame1) # 이미 있는 프레임 배열 모양에서 0으로 채워 반환
    hsv[..., 1] = 255
    # model.h5 모델로 사용자의 no_move, up_move, down_move를 구별하기 위한 변수들
    i = 0
    prediction_str = ""
    repetitions = 0
    up = 0
    down = 0
    no_move = 0
    current_move = 0
    initial = -1
    
    # 운동 횟수 카운터와 상태 표현 변수
    counter = 0
    stage = None
    
    while cap.isOpened():
        i += 1
        ret, frame2 = cap.read() # 웹캠에서 받아온 동영상을 프레임 단위로 나눈 것
        if not ret: break
        if cv2.waitKey(30) >= 0: break # 윈도우 종료에 대한 키값 설정
            
        # BGR을 RGB로 변환
        image = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # 포즈 검출
        results = pose.process(image)

        # 다시 BGR로 변환
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY) # prvs와 구분하여 파넬백 변환을 수행하기 위한 변수
        flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0) # 파넬백 광학 흐름 변환

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) # 좌표를 극좌표로 변환
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) # 정규화
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) # hsv 컬러공간을 BGR 컬러 공간으로 변환

        image2 = cv2.resize(rgb, (64, 64)) # 학습된 모델에 적용할 수 있도록 64X64크기로 조정
        image2 = image2.reshape((1,) + image2.shape)
        image2 = image2 / 255.0
        prediction = np.argmax(model.predict(image2), axis=-1)[0] # 확률값으로 나온 예측값에서 가장 큰 값의 인덱스 반환

        if prediction == 0: # 내려가고 있다고 예측
            down += 1
            if down == 3:
                if initial == -1:
                    initial = 0
                if current_move == 2: # 끝까지 올라온 상태에서 다시 내려가면 횟수 증가
                    counter += 1
                current_move = 0 # 현재 움직임은 '하강'
            elif down > 0:
                up = 0
                no_move = 0
        elif prediction == 2: # 올라가고 있다고 예측
            up += 1
            if up == 3 and initial != -1:
                current_move = 2 # 현재 움직임은 '상승'
            elif up > 1:
                down = 0
                no_move = 0
        else: # 움직임이 없다고 예측
            no_move += 1
            if no_move == 15:
                current_move = 1 # 현재 움직임은 '정지'
            elif no_move > 10:
                up = 0
                down = 0
        # 윈도우에 글자를 표현하기 위해 폰트, 색상, 위치, 크기, 타입 등을 설정
        font = cv2.FONT_HERSHEY_SIMPLEX 
        bottomLeftCornerOfText = (10, 400)
        fontScale = 1
        fontColor = (255, 255, 255)
        lineType = 5
        prvs = next

        # 랜드마크 검출
        try:
            landmarks = results.pose_landmarks.landmark

            # 좌측 지표값
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # 좌측 각도계산
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            left_waist_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            # 우측 지표값
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # 우측 각도계산
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            right_waist_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

            if ex == 1: # 팔굽혀펴기
                # stage 및 피드백
                if left_elbow_angle < 110:
                    stage = "down" # 상태가 내려가고 있다
                    # 리스트에 각각의 값들을 삽입
                    a.append(left_elbow_angle)
                    b.append(left_shoulder_angle)
                    c.append(right_elbow_angle)
                    d.append(right_shoulder_angle)
                if left_elbow_angle > 130 and stage == 'down':
                    stage = "up" # 올라온 상태
                    tmp = min(a) # 다 올라왔으므로 '하강' 일 때 찍은 최저점 반환
                    index = a.index(tmp) # 최저점의 배열인덱스 반환
                    if LR.predict([[tmp, b[index], c[index], d[index]]]) == 1: # 최저점에서 모델로 평가를 하니 잘한 동작이더라
                        print("잘했습니다")
                    else: # 최저점에서 모델로 평가를 하니 좀 더 숙여야 겠더라
                        print("더 숙이세요")
                    #배열 초기화
                    a.clear()
                    b.clear()
                    c.clear()
                    d.clear()
            else: # 스쿼트
                # stage 및 피드백
                # 위의 팔굽혀펴기와 동일한 내용
                if left_knee_angle < 110:
                    stage = "down"
                    a.append(left_waist_angle)
                    b.append(left_knee_angle)
                    c.append(right_waist_angle)
                    d.append(right_knee_angle)
                if left_knee_angle > 130 and stage == 'down':
                    stage = "up"
                    tmp = min(b)
                    index = b.index(tmp)
                    if LR2.predict([[a[index], tmp, c[index], d[index]]]) == 1:
                        print("잘했습니다")
                    else:
                        print("허리가 너무 굽었거나 자세를 더 낮춰야 합니다")
                    a.clear()
                    b.clear()
                    c.clear()
                    d.clear()
        except:
            pass

        # 스탯 박스 그리기
        cv2.rectangle(image, (0, 0), (265, 73), (245, 117, 16), -1)

        # 반복 횟수 그리기
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # 상태를 윈도우에 나타냄
        cv2.putText(image, 'STAGE', (100, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (95, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # 현재 어떤 운동을 수행해야 하는지 구분하는 변수
        step='push-up'
        if ex==-1:
            step='squat'
            
        # 현재 운동 나타내기
        cv2.putText(image, step,
                    (400, 45),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # 반복 횟수 10회라면 다음 운동으로 넘어가는
        if counter == 10:
            cv2.putText(image, 'Done!', (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow('Mediapipe Feed', image)
            cv2.waitKey(2) # 2초동안 머물렀다 다음으로 넘억가는
            time.sleep(2)
            ex = ex * -1 # 운동 종류 변환
            if ex == 1:
                cv2.putText(image, 'Push-Up Time!', (170, 300),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, 'Squat Time!', (170, 300),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
            counter = 0 # 카운터 초기화
            flag = False

        # 검출된 랜드마크 그리기
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL) # 윈도우 이름 및 사이즈 설정
        cv2.resizeWindow('Mediapipe Feed', 1000, 750) # 윈도우 크기 리사이즈
        cv2.imshow('Mediapipe Feed', image) # 윈도우 나타내기 (현재는 프레임 단위)
        
        # 프로그램 초기화면 구성 및 운동 변경 때의 waiting 값 설정
        if flag == False:
            if flag2 == False:
                cv2.putText(image, 'First, Push-Up Time!', (150, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('Mediapipe Feed', image)
            cv2.waitKey(2)
            time.sleep(2)
            flag = True
            flag2 = True

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release() # 웹캠 해제
    cv2.destroyAllWindows() # 윈도우 파괴

잘했습니다
더 숙이세요
더 숙이세요
잘했습니다
더 숙이세요


KeyboardInterrupt: 